# Purpose

This notebook tracks development of a wrapper for the kameleon software suite to support legacy ccmc models until python 3 model readers become available. 

This notebook should be executed from a python3 environment with kamodo installed.

In [1]:
!python --version

Python 3.7.3


In [2]:
from kamodo.readers._kameleon_kamodo import Kameleon

In [3]:
# may need to specify the full path to cdf, else file doesn't load correctly
fname = '/Users/apembrok/Work/Kameleon_Sample_Data/LFM/Nov2015.LFM.S1-OF_mhd_2002-09-30T03-16-00Z.cdf'

# python_path should point to a python2 executeable with kameleon installed
python_path = '/Users/apembrok/miniconda2/envs/klive/bin/python'
kameleon = Kameleon(fname, python_path, 'rho', 'p', 'bx','by','bz')

/Users/apembrok/Work/Kameleon_Sample_Data/LFM/Nov2015.LFM.S1-OF_mhd_2002-09-30T03-16-00Z.cdf opened


In [4]:
kameleon

Kameleon([(rho(xvec), <function interpolate(xvec)>),
          (rho, <function interpolate(xvec)>),
          (p(xvec), <function interpolate(xvec)>),
          (p, <function interpolate(xvec)>),
          (bx(xvec), <function interpolate(xvec)>),
          (bx, <function interpolate(xvec)>),
          (by(xvec), <function interpolate(xvec)>),
          (by, <function interpolate(xvec)>),
          (bz(xvec), <function interpolate(xvec)>),
          (bz, <function interpolate(xvec)>)])

In [5]:
from kamodo import gridify
import numpy as np
points = np.arange(12*3).reshape((-1,3))

In [6]:
kameleon.rho(points)

array([ 1.75048208e+00,  4.82881117e+00,  5.63864784e+01,  1.87258148e+01,
        1.88226109e+01,  1.85908566e+01,  1.87104950e+01,  1.86995926e+01,
        1.86710930e+01,  1.86786823e+01, -1.09951163e+12, -1.09951163e+12])

In [7]:
kameleon.bx([[-30, 0, 0], [-20, 0, 0]])

[-14.527422904968262, -11.133100509643555]

In [11]:
kameleon

Kameleon([(rho(xvec), <function interpolate(xvec)>),
          (rho, <function interpolate(xvec)>),
          (p(xvec), <function interpolate(xvec)>),
          (p, <function interpolate(xvec)>),
          (bx(xvec), <function interpolate(xvec)>),
          (bx, <function interpolate(xvec)>),
          (by(xvec), <function interpolate(xvec)>),
          (by, <function interpolate(xvec)>),
          (bz(xvec), <function interpolate(xvec)>),
          (bz, <function interpolate(xvec)>)])

In [12]:
kameleon['bz_ijk'] = gridify(kameleon.bz, 
                             x = np.linspace(-25, -10, 30),
                             y = np.linspace(-10, 10, 40), z = 0)
kameleon

Kameleon([(rho(xvec), <function interpolate(xvec)>),
          (rho, <function interpolate(xvec)>),
          (p(xvec), <function interpolate(xvec)>),
          (p, <function interpolate(xvec)>),
          (bx(xvec), <function interpolate(xvec)>),
          (bx, <function interpolate(xvec)>),
          (by(xvec), <function interpolate(xvec)>),
          (by, <function interpolate(xvec)>),
          (bz(xvec), <function interpolate(xvec)>),
          (bz, <function interpolate(xvec)>),
          (bz_ijk(x, y, z),
           <function wrapped(x=array([-25.        , -24.48275862, -23.96551724, -23.44827586,
       -22.93103448, -22.4137931 , -21.89655172, -21.37931034,
       -20.86206897, -20.34482759, -19.82758621, -19.31034483,
       -18.79310345, -18.27586207, -17.75862069, -17.24137931,
       -16.72413793, -16.20689655, -15.68965517, -15.17241379,
       -14.65517241, -14.13793103, -13.62068966, -13.10344828,
       -12.5862069 , -12.06896552, -11.55172414, -11.03448276,
       -10

In [13]:
fig = kameleon.plot('bz_ijk')
fig.layout.scene.aspectmode = 'data'

In [14]:
import plotly.io as pio

pio.write_image(fig, 'images/kameleon_sample_out.svg')

![kameleon_sample](images/kameleon_sample_out.svg?1)